<a href="https://colab.research.google.com/github/Eastonco/CS437/blob/main/H5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework Assignment #5**

Assigned: March 24, 2021

Due: April 12, 2021

Connor Easton - 11557902

---

This assignment consists of questions that require a short answer and one Python programming task. You can enter your answers and your code directly in a Colaboratory notebook and upload the **shareable** link for the your notebook as your homework submission.


---

#1.

 (12 points) In most real world scenarios, data contain outliers. When using a support vector machine, outliers can be dealt with using a soft margin, specified in a slightly different optimization problem shown in Equation 7.38 in the text and called a soft-margin SVM.

Intuitively, where does a data point lie relative to the margin when $\zeta_i = 0$? Is this data point classified correctly?

> point is touching hyperplane. Not classified correctly

Intuitively, where does a data point lie relative to the margin when $0 < \zeta_i \leq 1$? Is this data point classified correctly?

> point is very close to hyperplane. Not classified correctly

Intuitively, where does a data point lie relative to the margin when $\zeta_i > 1$? Is this data point classified correctly?

> point has good margin to hyperplane.  classified correctly
---

#2.

(12 points) Suppose the two-layer neural network shown below processes the input (0, 1, 1, 0). If the actual output should be 0.2, show step-by-step how the vector of weights *v* will be updated using backpropagation and $\eta = 0.2$.

![](https://drive.google.com/uc?id=1mLkFgXA0drWp6nYL50n0BZv2Z13EA9CN)

> https://drive.google.com/file/d/1OgMrFVzCdHPh7JNGHL5A8wep2HErgcBp/view?usp=sharing

---


#3. 

(8 points) Under which of these conditions does an ensemble classifier perform best? There can be more than one right answer, explain all of your responses.

- Low prediction correlation between base classifiers.
- High prediction correlation between base classifiers.
- Base classifiers have low variance.
- Base classifiers have high bias.
- Base classifiers have high variance.

> Ensemble classifiers perfrom best when: there is a high prediciton correlation between the base classifier, and when the base classifiers have low variance. The first is because when all the base classifiers are in agreement, there is a high probability that they are all in agreement to the correct classificaiton. The second is because the low variance will lead to more consistent results of the base classififers.

---

#4.

 
Test your neural network using the MNIST dataset. Information on loading and storing this handwritten-digit dataset can be found at https://scikit-learn.org/stable/auto_examples/linear_model/plot_sparse_logistic_regression_mnist.html. Only consider digit classes '0' (which you can map onto value -1) and '1' (which you can map onto value 1). Train the network on a randomly-selected 2/3 of the data points and test on the remaining 1/3. You can report mean squared error or accuracy for the test data for a minimum of 10 epochs.


In [ ]:
from math import exp
from random import random
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import numpy as np


def initialize_network(n_inputs, n_hidden, n_outputs):
  network = list()  # initialize weights to random number in [0..1]
  hidden_layer = [{'weights':[random() for i in range(n_inputs+1)]} for i in range(n_hidden)]
  network.append(hidden_layer)
  output_layer = [{'weights':[random() for i in range(n_hidden+1)]} for i in range(n_outputs)]
  network.append(output_layer)
  return network


def activate(weights, inputs):
  activation = weights[-1]   # bias
  for i in range(len(weights)-1):
    activation += weights[i] * inputs[i]
  return activation


def transfer(activation): # sigmoid function
  return 1.0 / (1.0 + exp(-activation))


"""
def transfer(activation): # tanh function
  return np.tanh(activation)
"""


def forward_propagate(network, X, y):
  inputs = X
  for layer in network:
    new_inputs = []
    for node in layer:
      activation = activate(node['weights'], X)
      node['output'] = transfer(activation)
      new_inputs.append(node['output']) # output of one node input to another
    inputs = new_inputs
  return inputs   # return output from last layer


def transfer_derivative(output): # derivative of sigmoid function
  return output * (1.0 - output)


"""
def transfer_derivative(output): # derivative of tanh function
  return 1.0 - np.tanh(output)**2
"""


def backward_propagate_error(network, expected):
  for i in reversed(range(len(network))):
    layer = network[i]
    errors = []
    if i != len(network)-1:
      for j in range(len(layer)):
        error = 0.0
        for node in network[i+1]:
          error += (node['weights'][j] * node['delta'])
        errors.append(error)
    else:
      for j in range(len(layer)):
        node = layer[j]
        errors.append(expected[j] - node['output'])
    for j in range(len(layer)):
      node = layer[j]
      node['delta'] = errors[j] * transfer_derivative(node['output'])


def update_weights(network, x, y, eta):
  for i in range(len(network)):
    inputs = x
    if i != 0:
      inputs = [node['output'] for node in network[i-1]]
    for node in network[i]:
      for j in range(len(inputs)):
         node['weights'][j] += eta * node['delta'] * inputs[j]
      node['weights'][-1] += eta * node['delta']


def train_network(network, X, y, eta, num_epochs, num_outputs):
  expected = np.full((2), 0)
  for epoch in range(num_epochs):
    sum_error = 0
    # There are two output nodes. The one corresponding to the correct label
    # should output 1, the other should output 0.
    for i in range(len(y)):
      outputs = forward_propagate(network, X[i], y[i])
      if y[i] == 0:
        expected[0] = 1
        expected[1] = 0
      else:
        expected[0] = 0
        expected[1] = 1
      sum_error += sum([(expected[i] - outputs[i])**2 for i in range(len(expected))])
      backward_propagate_error(network, expected)
      update_weights(network, X[i], y[i], eta)    
    print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, eta, sum_error))


def test_network(network, X, y, num_outputs):
  expected = np.full((2), 0)
  sum_error = 0
  # There are two output nodes. The one corresponding to the correct label
  # should output 1, the other should output 0.
  for i in range(len(y)):
    outputs = forward_propagate(network, X[i], y[i])
    if y[i] == 0:
      expected[0] = 1
      expected[1] = 0
    else:
      expected[0] = 0
      expected[1] = 1
    sum_error += sum([(expected[i] - outputs[i])**2 for i in range(len(expected))])
  print('mse of test data is', sum_error / float(len(y)))


if __name__ == "__main__":
  # Load data from https://www.openml.org/d/554
  features, targets = fetch_openml('mnist_784', version=1, return_X_y=True)
  X = []
  y = []
  for i in range(len(targets)):
    if targets[i] == '1' or targets[i] == '0':
      X.append(features[i])
      if targets[i] == '0':
        y.append(-1)
      else:
        y.append(1)
  n_inputs = len(X[0])
  n_outputs = 2  # possible class values are '0' and '1'
  # Create a network with 1 hidden layer containing 2 nodes
  network = initialize_network(n_inputs, 2, n_outputs)
  X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.67, test_size=0.33)
  # train network for 10 epochs using learning rate of 0.1 
  train_network(network, X_train, y_train, 0.5, 10, n_outputs)
  for layer in network:
    print('layer \n', layer)
  test_network(network, X_test, y_test, n_outputs)

>epoch=0, lrate=0.500, error=23.286
>epoch=1, lrate=0.500, error=1.436
>epoch=2, lrate=0.500, error=0.833
>epoch=3, lrate=0.500, error=0.588
>epoch=4, lrate=0.500, error=0.455
>epoch=5, lrate=0.500, error=0.371
>epoch=6, lrate=0.500, error=0.313
>epoch=7, lrate=0.500, error=0.271
>epoch=8, lrate=0.500, error=0.239
>epoch=9, lrate=0.500, error=0.214
layer 
 [{'weights': [0.7029649542989028, 0.48836349218356, 0.4236293203879681, 0.9676770039797569, 0.8962045475314404, 0.8774715266330528, 0.24280808623053185, 0.5549958950935572, 0.7217222535635968, 0.1402445997285776, 0.5313866707559138, 0.4902582289400752, 0.48956625477498483, 0.3773565815891282, 0.13105105552035823, 0.06531900597730322, 0.75686489873331, 0.558658411206024, 0.9955285581730597, 0.9705857479755139, 0.4945859190507179, 0.9547111770972951, 0.3084972098672142, 0.39619887688450184, 0.6470264164877908, 0.7524159532946243, 0.8954209404463926, 0.7603955264601431, 0.7237992149409537, 0.8862116115253971, 0.32874758090589296, 0.2176